# Jobs scraper
My intention with this project is to scrape jobs data from the web, and pick out trends, keywords etc, to help me optimise my job applications and CV for the jobs that are actually available

# Resources
- https://github.com/spinlud/py-linkedin-jobs-scraper existing python tool for scraping jobs data from linkedin.
- 



In [73]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import random


In [2]:
base_url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Science&location=Glasgow%2C%20Scotland%2C%20United%20Kingdom&geoId=102681496&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'


In [46]:
page = requests.get(base_url)

In [47]:
page

<Response [200]>

In [48]:
soup = BeautifulSoup(page.content, 'html.parser')

In [49]:
job_list = soup.find(class_='jobs-search__results-list')

In [55]:
job_listall = soup.find_all(class_='base-card')

In [ ]:
def get_page(url, delay_timer=None, delay=2, output=True):
    toc = time.time()
    if delay_timer is not None and toc-delay_timer <= delay:
        wait = delay-(toc-delay_timer)
        print(f'Required delay: {delay}s, current timer: {toc-delay_timer:.1f}s, waiting further {wait:.1f}s')
        time.sleep(wait)
    elif delay_timer is None:
        print(f'No delay timer provided, proceeding')
    else:
        print(f'Required delay: {delay}s, current timer: {toc-delay_timer:.1f}s, proceeding')
    
    tic = time.time()
    page = requests.get(url)
    toc = time.time()
    print(f'Request took {toc-tic:.2f}s')
    
    return page, tic

In [127]:
page, dt = get_page(base_url)

soup = BeautifulSoup(page.content, 'html.parser')

job_listall = soup.find_all(class_='base-card')
len(job_listall)

for i, card in enumerate(job_listall):
    if i>1: break
    
    this_jobname = card.find(class_='base-search-card__title').text.strip()
    print(this_jobname)
    
    link_tag = card.find(class_='base-card__full-link')
    this_url = link_tag.get('href')
    
    this_jobid = card.get('data-search-id')
    print(this_jobid)
    
    this_company = card.find(class_='job-search-card__subtitle').text.strip()
    print(this_company)
    
    this_companyurl = card.find(class_='job-search-card__subtitle').get('href')
    
    this_jobpostdate = card.find(class_=re.compile('job-search-card__listdate.*')).get('datetime')
    print(this_jobpostdate)
    
    this_joblocation = card.find(class_='job-search-card__location').text.strip()
    print(this_joblocation)
    
    # Request job page
    page_job, dt = get_page(this_url,dt)
    soup = BeautifulSoup(page_job.content,'html.parser')
    
    this_jobpagehtml = soup.find(class_='show-more-less-html__markup')
    print('Jobpage content extracted')
    
    this_jobpagehtmlstr = str(this_jobpagehtml)
    
    match = re.match('[\S\n\t\v ]*(Salary:)([\£\$\d, ]*)<br/>[\S\n\t\v ]*',this_jobpagehtmlstr)    
    if match is not None:
        print(match.group(1), match.group(2))
    else:
        print('No salary value detected')
        
    print('---')
    

No delay timer provided, proceeding
Request took 0.61s
Data Scientist
xs38N/JQdrgvBeKTrNPViQ==
Euro Garages Ltd
2021-06-05
Blackburn, Scotland, United Kingdom
Required delay: 2s, current timer: 0.7s, waiting further 1.3s
Request took 0.72s
Jobpage content extracted
Salary:  £38,000
---
Data Analyst
xs38N/JQdrgvBeKTrNPViQ==
AXA UK
2021-06-04
Glasgow, Scotland, United Kingdom
Required delay: 2s, current timer: 0.8s, waiting further 1.2s
Request took 0.72s
Jobpage content extracted
No salary value detected
---


In [60]:
card = job_list.find(class_='base-card')
print(card.prettify())

<div class="base-card base-card--link base-search-card base-search-card--link job-search-card" data-column="1" data-entity-urn="urn:li:jobPosting:2582560990" data-row="1" data-search-id="8csD+jAa44rTCq98U+fqeA==" data-tracking-id="lTo9JVd7dYQJqJHfYe6kEg==">
 <a class="base-card__full-link" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://uk.linkedin.com/jobs/view/data-analyst-at-axa-uk-2582560990?refId=8csD%2BjAa44rTCq98U%2BfqeA%3D%3D&amp;trackingId=lTo9JVd7dYQJqJHfYe6kEg%3D%3D&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_jserp-result_search-card">
  <span class="screen-reader-text">
   Data Analyst
  </span>
 </a>
 <div class="search-entity-media">
  <img alt="" class="artdeco-entity-image artdeco-entity-image--square-4" data-delayed-url="https://media-exp3.licdn.com/dms/image/C4D0BAQHIqvY5mdWEww/company-logo_100_100/0/1622532283499?e=1631145600&amp;v=beta&amp;t=d1Z1p8nJTrwlcugYDXj5avLF_N

In [19]:
link_tag = card.find(class_='base-card__full-link')
this_url = link_tag.get('href')
this_url

'https://uk.linkedin.com/jobs/view/data-analyst-at-axa-uk-2582560990?refId=kiWe4aTAtyAluSqGyj0iqQ%3D%3D&trackingId=Xv7hffDgaPCePR7hj8yrmw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'

In [21]:
this_jobid = card.get('data-search-id')
this_jobid

'kiWe4aTAtyAluSqGyj0iqQ=='

In [62]:
this_jobname = card.find(class_='base-search-card__title').text.strip()
this_jobname

'Data Science Consultant'

In [63]:
this_company = card.find(class_='job-search-card__subtitle').text.strip()
this_company

'Metrica Recruitment'

In [64]:
this_companyurl = card.find(class_='job-search-card__subtitle').get('href')
this_companyurl

'https://www.linkedin.com/company/metrica-recruitment?trk=public_jobs_jserp-result_job-search-card-subtitle'

In [68]:
this_jobpostdate = card.find(class_=re.compile('job-search-card__listdate.*')).get('datetime')
this_jobpostdate

'2021-03-12'

In [66]:
print(card.prettify())

<div class="base-card base-card--link base-search-card base-search-card--link job-search-card" data-column="1" data-entity-urn="urn:li:jobPosting:2458969679" data-row="2" data-search-id="8csD+jAa44rTCq98U+fqeA==" data-tracking-id="nS4Pvb+ieW4lb0LLaJaLOg==">
 <a class="base-card__full-link" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://uk.linkedin.com/jobs/view/data-science-consultant-at-metrica-recruitment-2458969679?refId=8csD%2BjAa44rTCq98U%2BfqeA%3D%3D&amp;trackingId=nS4Pvb%2BieW4lb0LLaJaLOg%3D%3D&amp;position=2&amp;pageNum=0&amp;trk=public_jobs_jserp-result_search-card">
  <span class="screen-reader-text">
   Data Science Consultant
  </span>
 </a>
 <div class="search-entity-media">
  <img alt="" class="artdeco-entity-image artdeco-entity-image--square-4" data-delayed-url="https://media-exp1.licdn.com/dms/image/C560BAQE-QWIVL-b-5A/company-logo_100_100/0/1519905145941?e=1631145600&amp;v=

In [33]:
this_joblocation = card.find(class_='job-search-card__location').text.strip()
this_joblocation

'Glasgow, Scotland, United Kingdom'

## Try opening job_url

In [37]:
page_job = requests.get(this_url)
soup = BeautifulSoup(page_job.content,'html.parser')

In [84]:
this_jobpagecontent = soup.find(class_='show-more-less-html__markup')
this_jobpagecontent

<div class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5">Are you intrigued by data? Yelp has hundreds of millions of pieces of user-contributed content, millions of users and business listings, and hundreds of thousands of advertising customers – and all of these numbers are constantly growing. Making sense of this data, deducing relationships between variables, and figuring out different interactions is hard work, but these insights are hugely impactful to Yelp’s business. Applied scientists uncover these insights through exploratory research and analysis, and carry the ideas all the way through to production-grade statistical or predictive models. They work in areas including pricing models and auction bidding strategies, learning to rank applications, personalized recommender systems, trust and safety / spam detection, and causal inference.<br/><br/>Yelp engineering culture is driven by our values: we’re a cooperative team that encourages individual authen

In [85]:
this_jobpagecontent.text

"Are you intrigued by data? Yelp has hundreds of millions of pieces of user-contributed content, millions of users and business listings, and hundreds of thousands of advertising customers – and all of these numbers are constantly growing. Making sense of this data, deducing relationships between variables, and figuring out different interactions is hard work, but these insights are hugely impactful to Yelp’s business. Applied scientists uncover these insights through exploratory research and analysis, and carry the ideas all the way through to production-grade statistical or predictive models. They work in areas including pricing models and auction bidding strategies, learning to rank applications, personalized recommender systems, trust and safety / spam detection, and causal inference.Yelp engineering culture is driven by our values: we’re a cooperative team that encourages individual authenticity and “unboring” solutions to problems. We enable all new team members to deploy working

# Selenium approach
I realised I can't get the job description from the HTML, so need a web-browser based scraping method. Using Selenium